# MongoDB Atlas Search: Synonym

## Setup

In [1]:
# import our libraries, instantiate our classes

import sys
import os
from pymongo import MongoClient
from dotenv import load_dotenv
import pprint

# Load environment variables from a .env file, overriding any existing variables in the environment.
# This is useful for not cluttering the code with sensitive information like API keys and database URIs.
load_dotenv(override=True)

MONGO_URI = os.environ["MONGO_URI"]

client = MongoClient(MONGO_URI)['sample_mflix']

pp = pprint.PrettyPrinter(indent=1)

### Synonym search

#### Atlas Search Index

```json

{
  "mappings": {
    "dynamic": true
  },
  "synonyms": [
    {
      "analyzer": "lucene.standard",
      "name": "mySynonyms",
      "source": {
        "collection": "synonyms"
      }
    }
  ]
}

```

#### Do a search without synonym inplace

In [4]:
pipeline = [
    {
        '$search': {
            'index': 'synonyms',
            'text': {
                'query': 'wakeful',
                'path': 'plot',
                'synonyms': 'mySynonyms'                
            }
        }
    },
    {
        '$project' : {
            '_id' : 0,
            'title' : 1,
            'plot' : 1
        }
    },
    {
        '$limit' : 3
    }
]

result = list(client['movies'].aggregate(pipeline))
pp.pprint(result)

[]


### Insert synonym

In [ ]:
synonym_doc = {
  "mappingType": "equivalent",
  "synonyms": [
    "insomniac",
    "sleepless",
    "wakeful"
  ]
}

result = client['synonyms'].insert_one(synonym_doc)
print("Inserted doc: " + str(result.inserted_id))

#### Run the same search again

In [ ]:
pipeline = [
    {
        '$search': {
            'index': 'synonyms',
            'text': {
                'query': 'wakeful',
                'path': 'plot',
                'synonyms': 'mySynonyms'                
            }
        }
    },
    {
        '$project' : {
            '_id' : 0,
            'title' : 1,
            'plot' : 1
        }
    },
    {
        '$limit' : 3
    }
]

result = list(client['movies'].aggregate(pipeline))
pp.pprint(result)

#### Clean up synonym

In [3]:
del_filter = { "synonyms": {"$in" : ["insomniac"]}}

result = client['synonyms'].delete_many(del_filter)
print("Deleted docs: " + str(result.deleted_count))

Deleted docs: 1
